# Classifying Toxic Comments

by: <a href="http://keithqu.com">Keith Qu</a><br>

Some natural language classification of toxic comments using logistic regression and also keras (running on tensorflow). This is a very broad run through ranging from basic linear methods, to modified linear methods, to deep learning.

Methods include logistic regression, NB-SVM, and CNN and RNN (bidirectional LSTM) with Keras on a TensorFlow backend.

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re, string
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc
from scipy import sparse
from sklearn.model_selection import train_test_split
%matplotlib inline

## First look with logit

Toxicity categories:
<ul>
<li><b>toxic</b></li>
<li><b>severe_toxic</b></li>
<li><b>obscene</b></li>
<li><b>threat</b></li>
<li><b>insult</b></li>
<li><b>identity_hate</b></li>
</ul>
The labels are fairly self-explanatory, but it also seems like there should be a lot of overlap between the categories, since everything that qualifies as severe_toxic, obscene, threat, insult or identity_hate should also be regular toxic as well.

It makes sense that the categories are not exclusive. So we can treat them like 6 different classification problems.

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
train.tail()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0
159570,fff46fc426af1f9a,"""\nAnd ... I really don't think you understand...",0,0,0,0,0,0


Let's take a look at some of our test comments.

In [4]:
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
test.loc[0].comment_text

"Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time.,"

This looks like we have some obscenity and insult, combined with a dash of identity hate.

They are also completely wrong, since 50 Cent $>$ Ja Rule any day. Well, maybe not his last album...

### Missing data?

Only a little.

In [6]:
test.fillna(' ',inplace=True)

### Vectorizing the comments

We'll do it by words and by character. Internet comments are a cesspool, and there are character ngrams that can have toxic meanings. Maybe we can also combine them.

In [7]:
def tok(s):
    return re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])').sub(r' \1 ',s).split()

In [8]:
words = TfidfVectorizer(ngram_range=(1,2), lowercase=True,
                        analyzer='word',stop_words='english',tokenizer=tok,
                        min_df=3,max_df=0.9, sublinear_tf=1, smooth_idf=1, dtype=np.float32,
                       strip_accents='unicode')
chars = TfidfVectorizer(ngram_range=(1,5), lowercase=True,
                        analyzer='char',min_df=3,
                        max_df=0.9, sublinear_tf=1,smooth_idf=1,dtype=np.float32,)

In [9]:
train_words = words.fit_transform(train['comment_text'])
train_chars = chars.fit_transform(train['comment_text'])

### Only words

In [10]:
X = sparse.csr_matrix(train_words)
cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y = train[cols]

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=10101)

In [12]:
logit = LogisticRegression(C=4, dual=True)
pred = np.zeros((X_test.shape[0],y_test.shape[1]))
for i,c in enumerate(cols):
    logit.fit(X_train,y_train[c])
    pred[:,i] = logit.predict(X_test)

In [13]:
for i,c in enumerate(cols):
    print('Confusion matrix for', c)
    print(confusion_matrix(y_test[c],pred[:,i]))

Confusion matrix for toxic
[[42996   244]
 [ 1767  2865]]
Confusion matrix for severe_toxic
[[47295    87]
 [  390   100]]
Confusion matrix for obscene
[[45167   148]
 [  892  1665]]
Confusion matrix for threat
[[47720    15]
 [  121    16]]
Confusion matrix for insult
[[45231   269]
 [ 1158  1214]]
Confusion matrix for identity_hate
[[47414    40]
 [  338    80]]


### Only characters

In [14]:
X_c = sparse.csr_matrix(train_chars)
X_train,X_test,y_train,y_test = train_test_split(X_c,y,test_size=0.3,random_state=10101)

In [15]:
pred = np.zeros((X_test.shape[0],y_test.shape[1]))
for i,c in enumerate(cols):
    logit.fit(X_train,y_train[c])
    pred[:,i] = logit.predict(X_test)

In [16]:
for i,c in enumerate(cols):
    print('Confusion matrix for', c)
    print(confusion_matrix(y_test[c],pred[:,i]))

Confusion matrix for toxic
[[42927   313]
 [ 1540  3092]]
Confusion matrix for severe_toxic
[[47284    98]
 [  372   118]]
Confusion matrix for obscene
[[45146   169]
 [  791  1766]]
Confusion matrix for threat
[[47727     8]
 [  109    28]]
Confusion matrix for insult
[[45212   288]
 [ 1006  1366]]
Confusion matrix for identity_hate
[[47417    37]
 [  315   103]]


Characterwise vectorization appears to have better results with ngrams of up to size 5, but this could vary with different splits.

### Combine words and characters
Horizontal stacking the word and character sets to create a larger blended dataset.

In [13]:
X2 = sparse.hstack([train_words,train_chars])

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X2,y,test_size=0.3,random_state=10101)

In [19]:
pred = np.zeros((X_test.shape[0],y_test.shape[1]))
for i,c in enumerate(cols):
    logit.fit(X_train,y_train[c])
    pred[:,i] = logit.predict(X_test)

In [20]:
for i,c in enumerate(cols):
    print('Confusion matrix for', c)
    print(confusion_matrix(y_test[c],pred[:,i]))

Confusion matrix for toxic
[[42935   305]
 [ 1472  3160]]
Confusion matrix for severe_toxic
[[47270   112]
 [  367   123]]
Confusion matrix for obscene
[[45128   187]
 [  757  1800]]
Confusion matrix for threat
[[47716    19]
 [  106    31]]
Confusion matrix for insult
[[45198   302]
 [  982  1390]]
Confusion matrix for identity_hate
[[47404    50]
 [  308   110]]


So putting character and string combinations together seem to give better results than they are separately. However, we can see that the model does best in identifying toxic, obscene and insult comments, and these are the ones that that are likely to have specific keywords associated with them. There would appear to be heavy subjectivity when it comes to what exactly constitutes severe toxicity. Threats are very context-sensitive and identity hate is a lot easier to identify for some groups than others.

It's also extremely memory-intensive for a machine with a very normal 16 GB of RAM.

## NB-SVM

<A href="https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf">Wang & Manning (2012)</a> find that Multinomial Naive Bayes performs better at classifying smaller snippets of text, while SVM is superior with full-length text. By combining the two models with linear interpolation, they create a new model that is robust for a wide variety of text.

There is a very helpful Python <a href="https://www.kaggle.com/jhoward/nb-svm-strong-linear-baseline-eda-0-052-lb">implementation</a> by Jeremy Howard.

In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string, re
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc
from scipy import sparse
from sklearn.model_selection import train_test_split
%matplotlib inline

In [24]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
test.fillna(' ',inplace=True)

In [25]:
# here's the main part of the implementation by jhoward mentioned above
def pr(y_i, y):
    p = train_words[y == y_i].sum(0)
    return (p+1)/((y==y_i).sum()+1)

# Get rid of the punctuation. Again, thanks to jhoward for this...
def tok(s):
    return re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])').sub(r' \1 ',s).split()

In [34]:
words = TfidfVectorizer(ngram_range=(1,2), lowercase=True,
                        analyzer='word',stop_words='english',tokenizer=tok,
                        min_df=3,max_df=0.9, sublinear_tf=1, smooth_idf=1, use_idf=1,
                       strip_accents='unicode')

In [35]:
train_words = words.fit_transform(train['comment_text'])
test_words = words.transform(test['comment_text'])

In [40]:
pred = np.zeros((test.shape[0],len(cols)))
for i,c in enumerate(cols):
    logit = LogisticRegression(C=4, dual=True)    
    r = np.log(pr(1,train[c].values)/pr(0,train[c].values))
    X_nb = train_words.multiply(r)
    logit.fit(X_nb,train[c].values)
    pred[:,i] = logit.predict_proba(test_words.multiply(r))[:,1]

In [41]:
submission = pd.read_csv('sample_submission.csv')
submission[cols] = pred
submission.to_csv('submission.csv',index=False)

This gives a score in the 0.07s (mean column-wise log loss), which is on the high side. But it was quick and painless; there wasn't any lemmatization, feature engineering, using toxic word dictionaries, spellchecking, or using existing repositories of vectorized text. So there's a lot of room for improvement.

In [42]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.999996,0.056502,0.999964,0.002058,0.986572,0.362120
1,0000247867823ef7,0.006075,0.000966,0.004201,0.000120,0.005512,0.000392
2,00013b17ad220c46,0.009947,0.000812,0.004720,0.000098,0.004124,0.000283
3,00017563c3f7919a,0.001160,0.000283,0.001051,0.000240,0.001028,0.000225
4,00017695ad8997eb,0.016037,0.000392,0.001696,0.000138,0.002533,0.000301


The first entry (0) is from the enlightened Ja Rule supporter shown above. We have detected the obscenity and insult (and so it is definitely toxic), but it's a bit weak on the identity hate measure. To be fair, calling someone a "fuckin white boy" ranks low on the identity hate ladder, but arguably it still should count. We don't really know what it's true classification is at this point.

## Keras/TensorFlow

The Keras API is a nice way to access TensorFlow, which will be needed to create convolutional (CNN) and recurrent neural networks (RNNs).

In [116]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

from keras.models import Sequential
from keras.layers import (Dense, Dropout, Input, LSTM, Activation, Flatten,
                          Convolution1D, MaxPooling1D, Bidirectional,
                         GlobalMaxPooling1D, Embedding)
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
test.fillna(' ',inplace=True)

Going to convert everything to lower case, remove stopwords, lemmatize words (get the root), and convert text into batch sequences of 200 length to run through the learning models.

While I don't do it here, using <a href="https://nlp.stanford.edu/projects/glove/">GloVe</a>'s pretrained Common Crawl or Twitter data could be very useful. Wikipedia word co-occurrence data is available as well, but Twitter or general crawls can give a better idea of words and word patterns in everyday use on the Internet.

In [3]:
def clean_up(t):
    t = t.strip().lower()
    words = t.split()
    
    # first get rid of the stopwords, or a lemmatized stopword might not
    # be recognized as a stopword
    
    imp_words = ' '.join(w for w in words if w not in set(stopwords.words('english')))

    # lemmatize based on adjectives (J), verbs (V), nouns (N) and adverbs (R) to
    # return only the base words (as opposed to stemming which can return
    # non-words). e.g. ponies -> poni with stemming, and pony with lemmatizing
    
    final_words = ''
    
    lemma = WordNetLemmatizer()
    for (w,tag) in pos_tag(word_tokenize(imp_words)):
        if tag.startswith('J'):
            final_words += ' '+ lemma.lemmatize(w, pos='a')
        elif tag.startswith('V'):
            final_words += ' '+ lemma.lemmatize(w, pos='v')
        elif tag.startswith('N'):
            final_words += ' '+ lemma.lemmatize(w, pos='n')
        elif tag.startswith('R'):
            final_words += ' '+ lemma.lemmatize(w, pos='r')
        else:
            final_words += ' '+ w
    
    return final_words

# what a great name. do_stuff

def do_stuff (df):
    text = df['comment_text'].copy()
    
    # First get rid of anything that's not a letter. This may not be the greatest idea, since
    # on3 c4n 3451ly substitute numbers in for letters, but keep it like this for now.
    
    text.replace(to_replace={r'[^\x00-\x7F]':' '},inplace=True,regex=True)
    text.replace(to_replace={r'[^a-zA-Z]': ' '},inplace=True,regex=True)
    
    # Then lower case, tokenize and lemmatize

    text = text.apply(lambda t:clean_up(t))
    return text

def tok_seq (train,test):
    tok = Tokenizer(num_words=20000)
    tok.fit_on_texts(x)
     
    # set our max text length to 200 characters
    seq_train = tok.texts_to_sequences(train)
    seq_test = tok.texts_to_sequences(test)    
    data_train = pad_sequences(seq_train,maxlen=200)
    data_test = pad_sequences(seq_test,maxlen=200)
    
    return data_train,data_test

Convolution model with 25% dropouts to help with generalization.

In [127]:
def seq_model (X_train, y_train, test, val='no'):
    model = Sequential()
    model.add(Embedding(20000,50,input_length=200))
    model.add(Dropout(0.25))
    model.add(Conv1D(250, activation='relu',padding='valid',kernel_size=3))
    model.add(GlobalMaxPooling1D())
    model.add(Dense(250))
    model.add(Dropout(0.25))
    model.add(Activation('relu'))
    
    # A sigmoid ensures a bounded solution in (0,1)    
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # batch_size=1000 seems to be the limit of my 2gb GTX 960m
    # like with all predictive modeling, there is an under/overfitting
    # tradeoff between too few epochs and too many
    if val == 'no':
        model.fit(X_train,y_train,batch_size=1000,epochs=5)
    else:
        model.fit(X_train,y_train,batch_size=1000,epochs=5,validation_split=0.1)
    pred = model.predict(test)
    return pred

Bidirectional LSTM model with similar dropouts.

In [128]:
def bidirect_model (X_train, y_train, test, val='no'):
    model=Sequential()
    model.add(Embedding(20000,100,input_length=200))
    model.add(Bidirectional(LSTM(50, return_sequences=True)))
    model.add(GlobalMaxPool1D())
    model.add(Dropout(0.25))
    model.add(Dense(250))
    model.add(Dropout(0.25))
    model.add(Activation('relu'))
    
    model.add(Dense(6,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    if val == 'no':
        model.fit(X_train,y_train,batch_size=1000,epochs=4)
    else:
        model.fit(X_train,y_train,batch_size=1000,epochs=4,validation_split=0.1)
    pred = model.predict(test)
    return pred

### Train/Validation

First let's take a look at results with a train test split.

In [4]:
X_train = do_stuff(train)

In [19]:
cols = ['toxic','severe_toxic','obscene','threat','insult','identity_hate']
y_train = train[cols].values

In [68]:
Xt, Xv, yt, yv = train_test_split(X_train, y_train, test_size=0.2, random_state=11)

In [69]:
Xt, Xv = tok_seq(Xt,Xv)

In [76]:
pred_seq = seq_model (Xt, yt, Xv)

Epoch 1/5
127656/127656 [==============================] - 27s 212us/step - loss: 0.1767 - acc: 0.9570
Epoch 2/5
127656/127656 [==============================] - 26s 205us/step - loss: 0.1059 - acc: 0.9648
Epoch 3/5
127656/127656 [==============================] - 26s 205us/step - loss: 0.0596 - acc: 0.9790
Epoch 4/5
127656/127656 [==============================] - 26s 205us/step - loss: 0.0474 - acc: 0.9822
Epoch 5/5
127656/127656 [==============================] - 26s 205us/step - loss: 0.0441 - acc: 0.9834


In [79]:
pred_bid = bidirect_model(Xt,yt,Xv)

Epoch 1/4
127656/127656 [==============================] - 161s 1ms/step - loss: 0.1832 - acc: 0.9599
Epoch 2/4
127656/127656 [==============================] - 158s 1ms/step - loss: 0.0664 - acc: 0.9770
Epoch 3/4
127656/127656 [==============================] - 159s 1ms/step - loss: 0.0478 - acc: 0.9823
Epoch 4/4
127656/127656 [==============================] - 158s 1ms/step - loss: 0.0436 - acc: 0.9834


Unsurprisingly, bidirectional adds a hefty amount of computing time.

Using a GPU for computation is kind of like reading Playboy for the articles, where it might seem questionable at first but GPUs are really good for deep learning and every now and then Playboy has a great article about the military industrial complex.

In [125]:
roc_auc_score(yv, pred_seq)

0.96476484985764843

In [126]:
roc_auc_score(yv, pred_bid)

0.96693766801110925

In [109]:
for i,c in enumerate(cols):
    print ('Correlation between results of', c)
    print(np.corrcoef(pred_bid[:,i],pred_seq[:,i])[0,1])

Correlation between results of toxic
0.936748623061
Correlation between results of severe_toxic
0.931061368507
Correlation between results of obscene
0.961609219062
Correlation between results of threat
0.840388029318
Correlation between results of insult
0.950607373316
Correlation between results of identity_hate
0.911772656275


The two sets of predictions are fairly highly correlated for our validation split, but we'll still try basic mean ensembling later anyway.

### Testing on the public test set

In [17]:
X_test = do_stuff(test)

In [80]:
X_train,X_test=tok_seq(X_train,X_test)

In [81]:
y_train = train[['toxic', 'severe_toxic','obscene','threat','insult','identity_hate']].values

In [129]:
finalpred_seq = seq_model(X_train, y_train, X_test,val='yes')

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 31s 219us/step - loss: 0.1726 - acc: 0.9565 - val_loss: 0.1125 - val_acc: 0.9627
Epoch 2/5
143613/143613 [==============================] - 30s 209us/step - loss: 0.0956 - acc: 0.9683 - val_loss: 0.0640 - val_acc: 0.9778
Epoch 3/5
143613/143613 [==============================] - 30s 209us/step - loss: 0.0526 - acc: 0.9809 - val_loss: 0.0568 - val_acc: 0.9793
Epoch 4/5
143613/143613 [==============================] - 30s 209us/step - loss: 0.0469 - acc: 0.9824 - val_loss: 0.0558 - val_acc: 0.9794
Epoch 5/5
143613/143613 [==============================] - 30s 209us/step - loss: 0.0439 - acc: 0.9834 - val_loss: 0.0561 - val_acc: 0.9796


In [131]:
finalpred_bid = bidirect_model (X_train, y_train, X_test,val='yes')

Train on 143613 samples, validate on 15958 samples
Epoch 1/4
143613/143613 [==============================] - 188s 1ms/step - loss: 0.1680 - acc: 0.9618 - val_loss: 0.0865 - val_acc: 0.9717
Epoch 2/4
143613/143613 [==============================] - 184s 1ms/step - loss: 0.0690 - acc: 0.9759 - val_loss: 0.0558 - val_acc: 0.9801
Epoch 3/4
143613/143613 [==============================] - 184s 1ms/step - loss: 0.0494 - acc: 0.9819 - val_loss: 0.0529 - val_acc: 0.9808
Epoch 4/4
143613/143613 [==============================] - 183s 1ms/step - loss: 0.0446 - acc: 0.9831 - val_loss: 0.0547 - val_acc: 0.9811


In [134]:
submission1 = pd.read_csv('sample_submission.csv')
submission1[cols] = finalpred_seq
submission1.to_csv('submission1.csv',index=False)

In [135]:
submission2 = pd.read_csv('sample_submission.csv')
submission2[cols] = finalpred_bid
submission2.to_csv('submission2.csv',index=False)

In [136]:
submission3 = pd.read_csv('sample_submission.csv')
submission3[cols] = 0.5*(submission1[cols] + submission2[cols])
submission3.to_csv('submission3.csv',index=False)

## Conclusion

The obvious next step would be to use existing vectorized co-occurence dictionaries such as GloVe or Facebook fastText. Spell checking and toxic word dictionaries may also be helpful. Possible features that can be created include the use of all caps, prevalence of symbols within the body of text or exlamation and question marks. These can be determined prior to detokenization/lemmatization/forced lowercasing. Comment length can also be useful; anecdotally, at times there seems to be a slight corelation between the length of a comment and how angry its writer is.

These will almost certainly be necessary for significant score improvements.